In [1]:
"""
clone and install PySDM dependencies in Colab
"""
import os, sys
if 'google.colab' in sys.modules:
    %cd /content
    if not os.path.isdir('PySDM'):
        !git clone --depth 1 https://github.com/atmos-cloud-sim-uj/PySDM.git
        %cd PySDM
    else:
        %cd PySDM
        !git pull
    !pip install -r requirements.txt
    !ldconfig

In [2]:
import os, sys
if 'google.colab' in sys.modules:
    %cd /content/PySDM
else:
    sys.path.insert(0, os.path.join(os.getcwd(), '../..'))

In [3]:
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.settings import Settings
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.simulation import Simulation
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.storage import Storage
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.netcdf_exporter import NetCDFExporter
from PySDM_examples.Arabas_et_al_2015_Figs_8_9.demo_plots import _TimeseriesPlot, _ImagePlot

from PySDM_examples.utils.temporary_file import TemporaryFile
from PySDM_examples.utils.show_plot import show_plot
from PySDM_examples.utils.widgets import display, HTML, FloatProgress

from PySDM.products.state import AerosolSpecificConcentration, ParticleMeanRadius

from PySDM.physics import si

import numpy as np
from scipy.io.netcdf import netcdf_file
from matplotlib import pyplot, rcParams
from matplotlib.animation import FuncAnimation
from matplotlib.gridspec import GridSpec

In [4]:
class ProgBarController:
    def __init__(self):
        self.progress = FloatProgress(value=0.0, min=0.0, max=1.0)
        self.panic = False

    def __enter__(self):
        self.set_percent(0)
        display(self.progress)
        
    def __exit__(self, *_):
        pass
    
    def set_percent(self, value):
        self.progress.value = value

In [15]:
settings = Settings()

products = [
    AerosolSpecificConcentration(radius_threshold=settings.aerosol_radius_threshold),
    ParticleMeanRadius()
]

settings.n_sd_per_gridbox = 16
settings.grid = (32, 32)
settings.dt = .1 * si.second
settings.n_spin_up = int(si.hour / settings.dt)
settings.n_steps = int(2 * settings.n_spin_up)
settings.outfreq = int(1 * si.minute / settings.dt)
settings.condensation_rtol_x = 1e-2
settings.condensation_rtol_thd = 1e-2
settings.adaptive = True

storage = Storage()
simulation = Simulation(settings, storage)
simulation.reinit(products)
simulation.run(ProgBarController())

FloatProgress(value=0.0, max=1.0)

In [16]:
temp_file = TemporaryFile('.nc')
exporter = NetCDFExporter(storage, settings, simulation, temp_file.absolute_path)
exporter.run(ProgBarController())

FloatProgress(value=0.0, max=1.0)

In [21]:
class temp_file:
    absolute_path = '/home/slayoo/devel/PySDM/PySDM_examples/utils/temporary_files/tmpb3ab8gcn.nc'

default_figsize = rcParams["figure.figsize"]
figsize = (2 * default_figsize[0], 1.2* default_figsize[1])
gs = GridSpec(nrows=13, ncols=14)

fig = pyplot.figure(figsize=figsize)
axs = (
    fig.add_subplot(gs[:gs.nrows-4, :gs.ncols//2]),
    fig.add_subplot(gs[:gs.nrows-4, gs.ncols//2:]),
    fig.add_subplot(gs[gs.nrows-2:,2:-1])
)

ncdf = netcdf_file(temp_file.absolute_path, mode='r', mmap=False)

plots = (
    _ImagePlot(fig, axs[0], grid=ncdf.grid, size=ncdf.size, product=products[0], show=False, lines=False),
    _ImagePlot(fig, axs[1], grid=ncdf.grid, size=ncdf.size, product=products[1], show=False, lines=False),
    _TimeseriesPlot(fig, axs[2], ncdf.variables['T'][:], show=False)
)

plots[2].ax.axvline(ncdf.n_spin_up * ncdf.dt)
plots[2].ax.set_ylim(0, .001)

interval = 100 #ms
frame_list = np.arange(ncdf.variables['T'].shape[0], dtype=np.int)

def update(frame_num):
    plots[0].update(ncdf.variables['n_a_mg'][frame_num], step=frame_num*ncdf.outfreq)
    plots[1].update(ncdf.variables['radius_m1'][frame_num], step=frame_num*ncdf.outfreq)
    plots[2].update(ncdf.variables['surf_precip'][0:frame_num+1])
    return (plots[0].im, plots[1].im, plots[2].timeseries)
pyplot.close(fig)

In [22]:
animation = FuncAnimation(fig, update, frames=frame_list, interval=interval, blit=False)

if 'CI' not in os.environ:
    display(HTML(animation.to_html5_video()))
    file = TemporaryFile('.mp4')
    animation.save(file.absolute_path)
    display(file.make_link_widget())

HTML(value='<video width="864" height="344" controls autoplay loop>\n  <source type="video/mp4" src="data:vide…

HTML(value="<a href='../utils/temporary_files/tmp8rlv7w4_.mp4' target='_blank'>../utils/temporary_files/tmp8rl…